In [1]:
import pandas as pd
import numpy as np

from IPython.display import display
import datetime
import plotly.express as px

In [2]:
matches_table = pd.read_csv('data/matches.csv')

In [3]:
display(matches_table)

,Date,Match Number,Stage,Player 1,Player 2,Score 1,Score 2
0,2023-11-18,1,Group,Evan Sooklal,Will Simpson,2,6
1,2023-11-18,2,Group,Paul Bartenfeld,Roman Ramirez,2,6
2,2023-11-18,3,Group,Roman Ramirez,Evan Sooklal,6,3
3,2023-11-18,4,Group,Will Simpson,Paul Bartenfeld,6,1
4,2023-11-18,5,Group,Paul Bartenfeld,Evan Sooklal,1,6
5,2023-11-18,6,Group,Will Simpson,Roman Ramirez,3,6
6,2023-11-18,7,Group,Will Simpson,Evan Sooklal,6,2
7,2023-11-18,8,Group,Roman Ramirez,Paul Bartenfeld,6,4
8,2023-11-18,9,Group,Evan Sooklal,Roman Ramirez,6,5
9,2023-11-18,10,Group,Paul Bartenfeld,Will Simpson,4,6


In [4]:
set_player1 = set(matches_table['Player 1'])
set_player2 = set(matches_table['Player 2'])
set_players = set_player1.union(set_player2)
num_players = len(set_players)
display(set_players)

{'Aaron Carter',
 'Evan Sooklal',
 'Paul Bartenfeld',
 'Roman Ramirez',
 'Will Simpson'}

In [5]:
# elo stuff
K = 31
base = 3

def expected_score(ratingA, ratingB, base=None):
    # game variables
    # base = 10
    normal_elo_difference = 100

    return (1 / (1 + np.power(base, (ratingB - ratingA) / normal_elo_difference)))

def rating_change(score, expected_score, K=None):
    # k-factor: determines how strongly a result affects the rating change
    # usually between 10 and 40, bit with a lot of games, we want to change it often
    # K = 32
    return K * (score - expected_score)

def generate_elo(K, base):
    # ELO INITIALIZATION
    starting_elo = 1200.0
    elo = dict.fromkeys(set_players, starting_elo)
    elo_time = np.zeros([len(matches_table) + 1, num_players])
    elo_time_table = pd.DataFrame(elo_time)
    elo_time_table.columns = sorted(elo)
    elo_time_table.replace(0, np.NaN, inplace=True)

    prev_elo_time_table = elo_time_table.copy(deep=True)
    d_elo_time_table = elo_time_table.copy(deep=True)
    exp_elo_time_table = elo_time_table.copy(deep=True)
    eaa_elo_time_table = elo_time_table.copy(deep=True)

    record_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]),dtype=object)
    record_table.index = list(set_players)
    record_table.columns = list(set_players)

    for c in record_table.columns:
        for r in record_table.index:
            record_table.at[r, c] = np.array([0, 0])

    elo_time_table.iloc[0,:] = starting_elo
    prev_elo_time_table.iloc[0,:] = starting_elo


    for (i, row) in matches_table.iterrows():
        elo_p1 = elo[row['Player 1']]
        elo_p2 = elo[row['Player 2']]

        prev_elo_time_table.loc[i + 1, row['Player 1']] = elo_p1
        prev_elo_time_table.loc[i + 1, row['Player 2']] = elo_p2
        
        win_prob_p1 = expected_score(elo_p1, elo_p2, base=base)
        win_prob_p2 = expected_score(elo_p2, elo_p1, base=base)
        
        exp_elo_time_table.loc[i + 1, row['Player 1']] = win_prob_p1
        exp_elo_time_table.loc[i + 1, row['Player 2']] = win_prob_p2

        rating_change_p1 = rating_change(row['Score 1'] > row['Score 2'], win_prob_p1, K=K)
        rating_change_p2 = rating_change(row['Score 2'] > row['Score 1'], win_prob_p2, K=K)

        if rating_change_p1 > 0:
            record_table.loc[row['Player 1'], row['Player 2']][0] += 1
            record_table.loc[row['Player 2'], row['Player 1']][1] += 1
        elif rating_change_p2 > 0:
            record_table.loc[row['Player 2'], row['Player 1']][0] += 1
            record_table.loc[row['Player 1'], row['Player 2']][1] += 1

        d_elo_time_table.loc[i + 1, row['Player 1']] = rating_change_p1
        d_elo_time_table.loc[i + 1, row['Player 2']] = rating_change_p2

        elo[row['Player 1']] += rating_change_p1
        elo[row['Player 2']] += rating_change_p2

        elo_time_table.loc[i + 1, row['Player 1']] = elo[row['Player 1']]
        elo_time_table.loc[i + 1, row['Player 2']] = elo[row['Player 2']]

    prev_elo_time_table = prev_elo_time_table.iloc[1:,:]
    elo_time_table = elo_time_table.iloc[1:,:]
    d_elo_time_table = d_elo_time_table.iloc[1:,:]
    exp_elo_time_table = exp_elo_time_table.iloc[1:,:]

    return (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, record_table)

(elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, record_table) = generate_elo(K=K,base=base)

prev_elo_time_table.to_excel('exports/prev_elo_time_table.xlsx')
d_elo_time_table.to_excel('exports/d_elo_time_table.xlsx')
elo_time_table.to_excel('exports/elo_time_table.xlsx')
exp_elo_time_table.to_excel('exports/exp_elo_time_table.xlsx')



In [6]:
import plotly.graph_objects as go
import plotly.colors as pc

fig = go.Figure()

# adding player
ranking = 1
trace_colors = pc.qualitative.Bold # Light24
for (i, (player, current_rating)) in enumerate(sorted(elo.items(), key=lambda x:x[1], reverse=True)):
    fig.add_trace(go.Scatter(
        x=elo_time_table.index,
        y=elo_time_table[player],
        name=f'#{ranking} ({current_rating:.0f}) {player}',
        mode='lines+markers',
        connectgaps=True,
        text=[
            f"<br><b>Pre-Game ELO: </b>{x[1]:.0f}<br><b>Win Probability: </b>{x[0]:.1%}<br><br><b>Change in ELO:</b> {x[2]:+.0f}<br>" for x in zip(
                exp_elo_time_table[player],
                prev_elo_time_table[player],
                d_elo_time_table[player]
            )
        ],
        line=dict(
            shape='hv',
            color=trace_colors[i % len(trace_colors)]
        )
    ))
    ranking += 1

# adding highlighting by tournament
tournaments = list(sorted(set(matches_table['Date'])))
vrect_colors = ['green', 'red', 'yellow', 'blue', 'orange']
for (i, tourney) in enumerate(tournaments):
    fig.add_vrect(
        annotation_text=tourney,
        annotation_position="top left",
        x0=matches_table['Date'][matches_table['Date'] == tourney].index[0] + 0.5,
        x1=matches_table['Date'][matches_table['Date'] == tourney].index[-1] + 1.5,
        fillcolor=vrect_colors[i % len(vrect_colors)],
        opacity=0.1,
        line_width=0
    )

fig.update_layout(
    title=f'<b>Pokémon Showdown ELO Rating System by Roman Ramirez</b><br><i>Updated {str(datetime.datetime.now().strftime("%A, %b %d, %Y %H:%M:%S"))} CT',
    xaxis_title='<b>Game Number</b>',
    yaxis_title='<b>ELO Rating</b>'
)

customdata = np.stack((
        list(matches_table['Player 1']),
        list(matches_table['Player 2']),
        list(matches_table['Score 1']),
        list(matches_table['Score 2']),
        matches_table['Date']
    ), axis=-1)
hovertemplate = (
    '<i>%{customdata[4]|%A, %B %d, %Y}, Game %{x}</i><br>' +
    '<b>%{fullData.name}</b><br><br>' + 
    '<b>%{customdata[0]} vs. %{customdata[1]}</b><br>' +
    '<b>Final Score:</b> %{customdata[2]}-%{customdata[3]}<br>' + 
    '%{text}' + 
    '<b>Post-Game ELO:</b> %{y:,.0f}<br>' +
    '<extra></extra>'
)

fig.update_traces(
    customdata=customdata,
    hovertemplate=hovertemplate,
    opacity=0.8,
    legendgrouptitle_text='<b>#<i>Rank</i> (<i>Current ELO</i>) <i>Player</i></b>'
)
fig.show()
fig.write_html("index.html")

In [7]:
# historical table, with record_table
win_pct_table = record_table.copy(deep=True)
text_record_table = record_table.copy(deep=True)

for c in win_pct_table.columns:
    for r in win_pct_table.index:
        if c == r:
            win_pct_table.loc[r, c] = np.NaN
            text_record_table.loc[r, c] = ""
        else:
            w, l = win_pct_table.loc[r, c]
            w_pct = (w) / (w + l) if (w + l) != 0 else 0
            win_pct_table.loc[r, c] = w_pct
            text_record_table.loc[r, c] = f"{w}-{l}<br>{w_pct:,.1%}"

display(record_table)
display(win_pct_table)
display(text_record_table)

fig = px.imshow(
    win_pct_table,
    labels=dict(
        x="Player B",
        y="Player A",
        color="Win Percentage"
    )
)
fig.update_layout(
    title="<b>Win Percentage</b>"
)
fig.update_traces(
    text=text_record_table,
    texttemplate='%{text}'
)
fig.show()
fig.write_html("figures/win_pct.html")

,Paul Bartenfeld,Roman Ramirez,Aaron Carter,Will Simpson,Evan Sooklal
Paul Bartenfeld,"[0, 0]","[2, 4]","[1, 3]","[0, 6]","[1, 5]"
Roman Ramirez,"[4, 2]","[0, 0]","[1, 3]","[6, 3]","[1, 5]"
Aaron Carter,"[3, 1]","[3, 1]","[0, 0]","[3, 1]","[5, 2]"
Will Simpson,"[6, 0]","[3, 6]","[1, 3]","[0, 0]","[6, 2]"
Evan Sooklal,"[5, 1]","[5, 1]","[2, 5]","[2, 6]","[0, 0]"


,Paul Bartenfeld,Roman Ramirez,Aaron Carter,Will Simpson,Evan Sooklal
Paul Bartenfeld,NaN,0.333333,0.25,0.0,0.166667
Roman Ramirez,0.666667,NaN,0.25,0.666667,0.166667
Aaron Carter,0.75,0.75,NaN,0.75,0.714286
Will Simpson,1.0,0.333333,0.25,NaN,0.75
Evan Sooklal,0.833333,0.833333,0.285714,0.25,NaN


,Paul Bartenfeld,Roman Ramirez,Aaron Carter,Will Simpson,Evan Sooklal
Paul Bartenfeld,,2-4<br>33.3%,1-3<br>25.0%,0-6<br>0.0%,1-5<br>16.7%
Roman Ramirez,4-2<br>66.7%,,1-3<br>25.0%,6-3<br>66.7%,1-5<br>16.7%
Aaron Carter,3-1<br>75.0%,3-1<br>75.0%,,3-1<br>75.0%,5-2<br>71.4%
Will Simpson,6-0<br>100.0%,3-6<br>33.3%,1-3<br>25.0%,,6-2<br>75.0%
Evan Sooklal,5-1<br>83.3%,5-1<br>83.3%,2-5<br>28.6%,2-6<br>25.0%,


In [8]:
# current win probabilities

win_prob_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]))
win_prob_table.index = list(set_players)
win_prob_table.columns = list(set_players)

for p1 in set_players:
    for p2 in set_players:
        if p1 == p2:
            win_prob_table.loc[p1, p2] = np.NaN
        else:
            win_prob_table.loc[p1, p2] = expected_score(elo[p1], elo[p2], base=base)

fig = px.imshow(
    win_prob_table,
    labels=dict(
        x="Player B",
        y="Player A",
        color="Win Probability"
    )
)
fig.update_layout(
    title="<b>Matchup Predictor</b>"
)
fig.update_traces(
    text=win_prob_table.applymap(lambda x: "" if np.isnan(x) else f"{x:,.1%}"),
    texttemplate="%{text}"
)
fig.show()
fig.write_html("figures/win_prob.html")

In [9]:
# difference in win percentage and matchup predictor
# positive number means overrated, negative number means underrated
# understanding: Player A is overrated/underrated against Player B

key_upset_table = win_pct_table - win_prob_table

fig = px.imshow(
    key_upset_table,
    labels=dict(
        x="Player B",
        y="Player A",
        color="Key Upset"
    )
)
fig.update_layout(
    title="<b>Overrated/Underrated Table</b>: High percentage means you historically outperform your ELO matchup."
)
fig.update_traces(
    text=key_upset_table.applymap(lambda x: "" if np.isnan(x) else f"{x:,.1%}"),
    texttemplate="%{text}"
)
fig.show()
fig.write_html("figures/overrated_underrated.html")

In [10]:
# system optimization

def optimization1():
    index = [_ for _ in np.arange(1, 51, 1.0)]
    columns = [_ for _ in np.arange(2, 11, 1.0)]

    opt_table = pd.DataFrame(np.zeros([len(index), len(columns)]))
    opt_table.index = index
    opt_table.columns = columns

    for base in opt_table.columns:
        for K in opt_table.index:
            (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, record_table) = generate_elo(K=K,base=base)
            num_upsets = 0
            for i in elo_time_table.index:
                prev_elo_slice = prev_elo_time_table.loc[i,:].dropna().values
                elo_slice = elo_time_table.loc[i,:].dropna().values
                combo_elo = np.array([prev_elo_slice, elo_slice])
                is_upset = ((combo_elo[1, 0] - combo_elo[0, 0]) * (combo_elo[0, 1] - combo_elo[0, 0])) > 0
                if is_upset:
                    num_upsets += 1
            opt_table.loc[K,base] = num_upsets

    fig = px.imshow(opt_table.T,labels=dict(y="Base", x="K-value", color="Number of Upsets"))#, text_auto=True, aspect="auto")
    fig.update_layout(
        title="<b>Total Number of Upsets against K-Value and Base</b>"
    )
    fig.show()
    return opt_table
opt1 = optimization1()
fig.write_html("figures/opt1.html")

In [11]:
# system optimization

def optimization2():
    index = [_ for _ in np.arange(1, 51, 1.0)]
    columns = [_ for _ in np.arange(2, 11, 1.0)]

    opt_table = pd.DataFrame(np.zeros([len(index), len(columns)]))
    opt_table.index = index
    opt_table.columns = columns

    for base in opt_table.columns:
        for K in opt_table.index:
            (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, record_table) = generate_elo(K=K,base=base)
            total_correctness = 0
            for i in elo_time_table.index:
                d_elo_slice = d_elo_time_table.loc[i,:].dropna().values
                exp_elo_slice = exp_elo_time_table.loc[i,:].dropna().values
                correctness = (np.sign(d_elo_slice) * exp_elo_slice).sum()
                total_correctness += correctness
            opt_table.loc[K,base] = total_correctness

    fig = px.imshow(opt_table.T,labels=dict(y="Base", x="K-value", color="Total Correctness"))#, text_auto=False, aspect="auto")
    fig.update_layout(
        title="<b>Win Percentage Correctness against K-Value and Base</b>"
    )
    fig.show()
    return opt_table
opt2 = optimization2()
fig.write_html("figures/opt2.html")

In [12]:
def norm_minmax(dataframe):
    return ((dataframe - dataframe.min(axis=None)) / (dataframe.max(axis=None) - dataframe.min(axis=None)))

norm_opt1 = norm_minmax(opt1)
norm_opt2 = norm_minmax(opt2)

fig = px.imshow(norm_opt2.T - norm_opt1.T,labels=dict(y="Base", x="K-value", color="Normalized Optimization"), text_auto=False, aspect="auto")
fig.update_layout(
    title="<b>K Value and Base against Both Optimization Functions</b>"
)
fig.show()
fig.write_html("figures/opt_total.html")